In [1]:
is_startup = True

In [2]:
if is_startup:
  !pip install transformers[torch]
  !pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 8.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_path = "/content/drive/MyDrive/Detalytics/chatbot/"
sys.path.insert(0, project_path)

Mounted at /content/drive


In [4]:
import os
if is_startup:
  os.chdir(project_path)
  print (os.getcwd())
  !pip install -e .
  !pip install -e alutils

/content/drive/MyDrive/Detalytics/chatbot
Obtaining file:///content/drive/MyDrive/Detalytics/chatbot
  Preparing metadata (setup.py) ... done
  Running setup.py develop for chatbot
Obtaining file:///content/drive/MyDrive/Detalytics/chatbot/alutils
  Preparing metadata (setup.py) ... done
  Running setup.py develop for alutils


In [39]:
test_dialogs = [
    "Do you have any pets?",
    "Do you have dogs?",
    "What is your favorite movie?",
    "do you like spending your (or not so) money?",
    "I am from the south, where are you from?",
    "what are u gonna do this weekend?",
    "I do not like sports.",
    "Do you have any hobbies ?",
    "I like to go to the gym.",
    "i like to eat fish",
    "Hello there!",
    "How are you?",
    "Tell me a fun fact about space.",
    "Can you recommend a good book to read?",
    "What are the health benefits of eating bananas?",
    "How can I make a chocolate cake?",
    "What's the weather like today?",
    "Can you recommend a good restaurant nearby?",
    "What are the benefits of exercise?",
    "How do I reset my email password?"
]

In [40]:
from torch.utils.data import Dataset
import json
import random

class ChatData(Dataset):
    def __init__(self, path:str, tokenizer, split='train'):
        self.data = json.load(open(path, "r"))

        X = []
        for dialog in self.data:
            texts = []
            last_sender = None
            for turn in dialog['dialog']:
                this_sender = turn['sender']
                if this_sender != last_sender:
                    texts.append(turn['text'])
                else:
                    texts[-1] += turn['text']
                last_sender = this_sender

            #add texts to self.X
            for i in range(len(texts)-1):
                X.append("<startofstring> " + texts[i] +" <bot>: "+ texts[i+1] + " <endofstring>")

        random.seed(259)
        random.shuffle(X)
        num_train = len(X) // 5 * 4

        if split == 'train':
            self.X = X[:num_train]
        else:
            self.X = X[num_train:]

        print ("Number of dialogs: ", len(self.X))
        for one_turn_dialog in self.X[:10]:
            print(one_turn_dialog)

        self.X_encoded = tokenizer(self.X, max_length=64, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

        output_path = f"{constants.DATA_PATH}convai_{split}.txt"
        output_file = open(output_path, 'w', encoding="utf-8")
        for text in self.X:
            output_file.write(f"{text}\n")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [41]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

In [47]:
from chatbot import constants

def train(train_loader, val_loader, model, optim, num_epochs):
    min_val_loss = 1e6
    test_epoch = 8
    log_path = f"{constants.RESULT_PATH}programming_hut.txt"
    log_file = open(log_path, 'w', encoding="utf-8")
    for num_epoch in tqdm.tqdm(range(num_epochs)):
        #training
        avg_loss = []
        model.train()
        for X, a in train_loader:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            avg_loss.append(loss.item())
            loss.backward()
            optim.step()
        train_loss = sum(avg_loss) / len(avg_loss)

        #validation
        avg_loss = []
        model.eval()
        for X, a in val_loader:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            avg_loss.append(loss.item())
            loss.backward()
            optim.step()
        val_loss = sum(avg_loss) / len(avg_loss)

        #epoch summary
        print (f"Epoch {num_epoch+1}: train_loss = {train_loss:.4f}, val_loss = {val_loss:.4f}")

        #save epoch
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            torch.save(model.state_dict(), f"{constants.CKPT_PATH}programming_hut/model_state.pt")

        if (num_epoch + 1) % test_epoch == 0:
            response = "------------------------\n"
            for prompt in test_dialogs:
                response_ = infer(prompt)
                response_ = response_.split('<bot>:')[-1].split('<endofstring>')[0]
                response += f"user: {prompt} \n bot: {response_} \n\n"
            log_file.write(response)
            log_file.flush()


def infer(prompt):
    prompt = f"<startofstring> {prompt} <bot>: "
    input = tokenizer(prompt, return_tensors="pt")
    X = input["input_ids"].to(device)
    a = input["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output

In [48]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

1

In [ ]:
import warnings
warnings.filterwarnings('ignore')

num_epochs = 256
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.to(device)
# model.eval()
# for query in test_dialogs:
#     print(infer(query))
model.resize_token_embeddings(len(tokenizer))

train_dataset = ChatData(f"{constants.DATA_PATH}convai_chat_data.json", tokenizer, split='train')
train_loader =  DataLoader(train_dataset, batch_size=32)
val_dataset = ChatData(f"{constants.DATA_PATH}convai_chat_data.json", tokenizer, split='val')
val_loader =  DataLoader(val_dataset, batch_size=8)

optim = Adam(model.parameters(), lr=1e-4)

print("training .... ")
train(train_loader, val_loader, model, optim, num_epochs)

Number of dialogs:  2376
<startofstring> i am a huge gamer <bot>: ok <endofstring>
<startofstring> No!  You don't behave, you go in the bad girl closet. <bot>: I'm just a little girl <endofstring>
<startofstring> On Monday, I'm going to have to tell my kindergarten class, who I teach not to run with scissors, that my fiance ran me through with a frickin' broadsword. <bot>: I like gray color <endofstring>
<startofstring> I do not like swimming, but i do like to swim <bot>: and what about the beach, sand? <endofstring>
<startofstring> where are you from? <bot>: I live in the midwest <endofstring>
<startofstring> You are stupid <bot>: I am not sure what you mean. <endofstring>
<startofstring> i am a cali . cali . <bot>: nice to meet you, i'm iza <endofstring>
<startofstring> What do you like to read <bot>: i read a lot , especially when i have time . <endofstring>
<startofstring> Are you going <bot>: Ha Na, I will go to your school tomorrow. <endofstring>
<startofstring> I don't understan

  0%|          | 0/256 [00:00<?, ?it/s]

Epoch 1: train_loss = 3.2317, val_loss = 1.1197


  0%|          | 1/256 [00:14<59:52, 14.09s/it]

Epoch 2: train_loss = 1.0962, val_loss = 0.8524


...

100%|█████████▉| 255/256 [56:59<00:13, 13.04s/it]

Epoch 255: train_loss = 0.1367, val_loss = 0.1092


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Epoch 256: train_loss = 0.1373, val_loss = 0.1082


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene